In [3]:
import pandas as pd

# Path to the downloaded file in Kaggle environment
file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
print("Before date conversion:")
print(data.head())

# Convert the 'Datetime' column to datetime
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Filter the data to keep only rows from the year 2010
data = data[data['Datetime'].dt.year >= 2010]

# Format the 'Datetime' column to the desired format: %Y-%m-%d %H:%M
data['Datetime'] = data['Datetime'].dt.strftime('%Y-%m-%d %H:%M')

# Display the first few rows after filtering and conversion
print("After filtering for >=2010 and date conversion:")
print(data.head())

# Save the modified DataFrame back to a new CSV file
output_file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv"
data.to_csv(output_file_path, index=False)

# Confirm that the file has been saved
print(f"Modified file saved as {output_file_path}")

Before date conversion:
              Datetime  ABI   ACT  AMA  ALB  JFK  LGA
0  1949-01-01 00:00:00  6.1  10.6  2.2 -3.3 -3.3 -3.9
1  1949-01-01 01:00:00  3.3   6.7  1.1 -3.3 -3.3 -3.9
2  1949-01-01 02:00:00  2.2   5.6  0.6 -3.3 -3.3 -3.9
3  1949-01-01 03:00:00  1.7   4.4  0.0 -2.8 -3.3 -3.9
4  1949-01-01 04:00:00  1.7   3.9  1.1 -2.8 -3.3 -3.9
After filtering for >=2010 and date conversion:
                Datetime  ABI  ACT  AMA  ALB  JFK  LGA
470398  2010-01-01 00:00  1.1  6.7 -0.6 -1.7  1.1  1.1
470399  2010-01-01 01:00  1.1  6.1 -3.3 -1.7  1.1  1.1
470400  2010-01-01 02:00  1.7  5.6 -3.9 -1.7  0.6  1.1
470401  2010-01-01 03:00  1.1  5.0 -3.9 -1.7  0.6  1.1
470402  2010-01-01 04:00  1.1  4.4 -5.0 -1.1  0.6  0.6
Modified file saved as S:\spatiotemporal-analysis\spacetimeformer-main\spacetimeformer\data\temperature-v1_modified.csv


!python S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\train.py spacetimeformer asos --context_points 128 --target_points 40 --run_name spacetimeformer_asos --batch_size 32 --warmup_steps 1000 --d_model 200 --d_ff 700 --enc_layers 1 --dec_layers 1 --dropout_emb .1 --dropout_ff .3 --d_qk 30 --d_v 30 --n_heads 10 --patience 10 --decay_factor .8 --data_path S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv

In [1]:
import pandas as pd

def subset_csv(src_path: str, dest_path: str, columns: list):
    """
    Reads a CSV file from src_path, extracts the specified columns, and writes the subset to dest_path.
    
    Parameters:
        src_path (str): Path to the source CSV file.
        dest_path (str): Path to save the subset CSV file.
        columns (list): List of column names to extract from the source CSV.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(src_path)
    
    # Create a subset of the DataFrame with the specified columns
    subset = df[columns]
    
    # Save the subset to the destination CSV file
    subset.to_csv(dest_path, index=False)
    print(f"Subset saved to {dest_path}")

In [4]:
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_VIC", "RRP_VIC",])
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD",])

Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv
Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv


In [5]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_VIC,RRP_VIC
0,2015-02-01 00:00,6529.375000,19.210000,4485.130000,14.590000
1,2015-02-01 01:00,6145.710000,18.195000,4055.830000,12.925000
2,2015-02-01 02:00,5782.555000,16.450000,3701.360000,10.440000
3,2015-02-01 03:00,5600.055000,14.200000,3488.495000,6.660000
4,2015-02-01 04:00,5619.520000,15.955000,3455.595000,4.380000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,5107.343333,93.497500
88101,2025-02-18 21:00,7653.483333,100.095833,4648.734167,87.070833
88102,2025-02-18 22:00,7361.791667,97.980000,4341.420833,84.615000
88103,2025-02-18 23:00,7002.186667,96.224167,4404.169167,86.157500


In [6]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_QLD,RRP_QLD
0,2015-02-01 00:00,6529.375000,19.210000,5687.755000,17.855000
1,2015-02-01 01:00,6145.710000,18.195000,5349.120000,16.100000
2,2015-02-01 02:00,5782.555000,16.450000,5148.620000,14.420000
3,2015-02-01 03:00,5600.055000,14.200000,5045.510000,13.025000
4,2015-02-01 04:00,5619.520000,15.955000,5066.075000,14.620000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,8026.143333,112.365000
88101,2025-02-18 21:00,7653.483333,100.095833,7663.199167,102.720833
88102,2025-02-18 22:00,7361.791667,97.980000,7187.470833,102.015000
88103,2025-02-18 23:00,7002.186667,96.224167,6736.655000,100.227500


In [2]:
import pandas as pd

def filter_columns_and_save(input_file_path, selected_columns, output_file_path):
    # Read the original dataframe from the input CSV file
    df = pd.read_csv(input_file_path)
    
    # Filter the dataframe to only include the selected columns
    df_filtered = df[selected_columns]
    
    # Save the filtered dataframe to the specified output file path
    df_filtered.to_csv(output_file_path, index=False)
    print(f"Filtered dataframe saved to {output_file_path}")

In [3]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic.csv


In [4]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC", "TOTALDEMAND_TAS", "RRP_TAS"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic_tas.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic_tas.csv
